In [ ]:
from processing_arithmetics.arithmetics.treebanks import treebank
from processing_arithmetics.arithmetics.MathExpression import MathExpression
from processing_arithmetics.arithmetics.MathTreebank import MathTreebank
from processing_arithmetics.sequential.architectures import ScalarPrediction
import numpy as np
import pandas as pd
from plotnine import *
import plotnine


# Initialise

load the two best trained models

In [ ]:
basename = "2018-10-11T11+02:00"
digits = np.arange(-10,11)
operators = ['+', '-']
arch1 = ScalarPrediction(digits=digits, operators=operators)
arch1.add_pretrained_model(model="./ScalarPrediction_GRU_infix_{}_10.h5".format(basename))
arch1.model.compile(
        loss=arch1.loss_functions,
        optimizer='adam',
        metrics=arch1.metrics,
        loss_weights=arch1.loss_weights
    )

arch2 = ScalarPrediction(digits=digits, operators=operators)
arch2.add_pretrained_model(model="./ScalarPrediction_GRU_infix_{}_16.h5".format(basename))
arch2.model.compile(
        loss=arch2.loss_functions,
        optimizer='adam',
        metrics=arch2.metrics,
        loss_weights=arch2.loss_weights
    )


# Build a set of expressions to evaluate the models on

In [ ]:
#tb = treebank(seed=0, kind='test', debug=True)
#tb = dict(tb)
tb = MathTreebank({L:1000 for L in ['L1','L2','L3', 'L4','L5','L6','L7', 'L8', 'L9']}, digits=digits)

In [ ]:
tb.examples[4][0]

In [ ]:
data = arch1.generate_test_data(tb, digits=digits)
results = []
for name, X, Y in data:
    for n, arch in [("10", arch1), ("16", arch2)]:
        result = arch.model.predict(X)
        results.append(result)
        #print "MODEL: ", n
        #print "\texpected:", Y['output']
        #print "\tresult:", result

In [ ]:
expression_lengths = []
max_minus_depths = []
depths = []
mode_switches = []
for example in tb.examples:
    symbols = example[0].iterate('infix')
    length = example[0].get_digit_positions(symbols).sum()
    for n in xrange(1,10):
        try:
            count = example[0].get_minus_depths(n).sum()
        except KeyError:
            break
        if count == 0:
            break
    max_minus_depths.append(n - 1)
    expression_lengths.append(length)
    depths.append(max(example[0].get_depths()))
    mode_list = example[0].get_modes()
    mode_switches.append(sum([1 if x != mode_list[n] else 0 for n, x in enumerate(mode_list[1:])]))

In [ ]:
plot_df = pd.concat([pd.DataFrame({
        'model': 10,
        'prediction': results[0].reshape([len(Y['output'])]),
        'expected': Y['output'],
        'length': expression_lengths,
        'minus_depth': max_minus_depths,
        'depth': depths,
        'mode_switch': mode_switches,
    }),pd.DataFrame({
        'model': 16,
        'prediction': results[1].reshape([len(Y['output'])]),
        'expected': Y['output'],
        'length': expression_lengths,
        'minus_depth': max_minus_depths,
        'depth': depths,
        'mode_switch': mode_switches,
    })])
plot_df['delta'] = plot_df['prediction'] - plot_df['expected']
plot_df['abs_delta'] = plot_df['delta'].abs()
with pd.option_context('max_rows', 2000):
    display(plot_df)

# Plot the actual expression result against the prediction/error

Below are some plots that try to figure out what the model is good and bad at.

In [ ]:
plotnine.options.figure_size = (16, 20)
ggplot(plot_df, aes(x="expected", y="prediction")) + \
    geom_jitter(aes(color='minus_depth'), size=1, alpha=0.4, width=0, height=0) + \
    facet_wrap("~ model", ncol=1) + \
    theme_minimal() + \
    theme(panel_background=element_rect(fill='#222222'),legend_position="top")

In [ ]:
plotnine.options.figure_size = (16, 20)
ggplot(plot_df, aes(x="expected", y="abs_delta")) + \
    geom_jitter(aes(color='minus_depth'), alpha=0.3, width=0.2, height=0) + \
    facet_wrap("~ model", ncol=1) + \
    scale_y_sqrt() + \
    theme_minimal() + \
    theme(panel_background=element_rect(fill='#222222'),legend_position="top")

In [ ]:
plotnine.options.figure_size = (12, 20)
ggplot(plot_df, aes(x="expected", y="delta")) + \
    geom_point(aes(color='length'), alpha=0.05) + \
    facet_wrap("~ model", ncol=1) + \
    theme_minimal() + \
    theme(panel_background=element_rect(fill='#222222'),legend_position="top")

In [ ]:
plotnine.options.figure_size = (12, 20)
ggplot(plot_df, aes(x="expected", y="delta")) + \
    geom_point(aes(color='depth'), alpha=0.05) + \
    facet_wrap("~ model", ncol=1) + \
    theme_minimal() + \
    theme(panel_background=element_rect(fill='#222222'),legend_position="top")

In [ ]:
plotnine.options.figure_size = (12, 20)
ggplot(plot_df, aes(x="expected", y="delta")) + \
    geom_point(aes(color='mode_switch'), alpha=0.05) + \
    facet_wrap("~ model", ncol=1) + \
    theme_minimal() + \
    theme(panel_background=element_rect(fill='#222222'),legend_position="top")